
# Projeto nº 1 - Puzzle dos Robôs
## Introdução à Inteligência Artificial - edição 2021/22



<img src="../res/robots.png" alt="Drawing" style="width: 250px;"/>

## Grupo: 48

### Elementos do Grupo


Número: 55386          Nome: Vicente Sousa

Número: 54394          Nome: Afonso Esteves

Número: 54476          Nome: João Anjos


### Representação dos estados 
A representação dos estados é feita através de 3 parâmetros, o size (tamanho do tabuleiro), blacks (lista com as coordenadas dos robôs pretos) e white (coordenadas do robô branco).
As coordenadas são representadas através de um tuplo de inteiros no formato (x,y), sendo que estes valores, seguindo a ideia de um referencial, teem origem no canto inferior esquerdo do tabuleiro. Segue-se uma imagem que representa este sistema de coordenadas, incluindo a coordenada do robô branco, para uma melhor compreensão:

<img src="../res/robotsaxis.png" alt="Drawing" style="width: 250px;"/>

In [1]:
#Código que representa um estado
class PuzzleRobotsState:
    def __init__(self, size, blacks, white):
        #Size of the board
        self.size = size
        #List of the coordinates of all black robotss
        self.blacks = blacks
        #white robot coordinates
        self.white = white

    def clone(self):
        return PuzzleRobotsState(self.size, list(self.blacks), self.white)

    def display(self):
        lowbar = '+' + '---+'*self.size
        print(lowbar)
        for row in range(self.size):
            print('| ', end = '')
            for col in range(self.size):
                cont = ' '
                coords = (col, 4-row)
                if col == row and col == self.size//2 : cont = 'X'
                elif coords in self.blacks: cont = 'B'
                elif coords == self.white: cont = 'W'
                print(cont +' | ', end = '')
            print('\n' + lowbar)

    def __eq__(self, obj):
        return isinstance(obj, PuzzleRobotsState) and self.size == obj.size and \
            self.white == obj.white and self.blacks == obj.blacks

    def __ne__(self, obj):
        return not self == obj
    
    def __hash__(self):
        return hash( (self.size, tuple(self.blacks), self.white) )


In [2]:
# Representação do estado com N=5 ilustrado no enunciado
PuzzleRobotsState(5, [(3,4), (0,2), (1,1), (3,1), (4,0)], (1,4))

In [3]:
# Representação de um estado com N=7 à sua escolha
PuzzleRobotsState(7, [(0,2), (1,6), (2,0), (2,5), (4,4), (5,2), (6,6)], (5,6))

### Representação das acções

As ações admissíveis são representadas através da classe PuzzleRobotsAction. Esta contém dois atributos, o moving que guarda o par ordenado (x,y), as coordenadas da peça que se vai mover, e o stop que guarda o par ordenado (x,y), as coordenadas da peça contra qual o moving vai embater. É ainda definido o método getFinalPos que utiliza lazy evaluation para calcular a posição final da peça que se move e o método actionCost que devolve o custo da ação executada. Abaixo encontra-se o código para a classe descrita juntamente com a implementação do PuzzleRobots.

In [4]:
from searchPlus import *

def tupleSub(first, *others):
    temp = list(first)
    for tuple_ in others:
        for i in range(min(len(temp), len(tuple_))):
            temp[i] -= tuple_[i]
    return tuple(temp)

class PuzzleRobotsAction:
    def __init__(self, moving, stop):
        #Coordinates of the moving piece
        self.moving = moving
        #Coordinates of the piece stoping the movement
        self.stop = stop
        #Final Position
        self.final_pos = ()

    def display(self):
        print("from: " + str(self.moving[0])+", "+str(self.moving[1])+" colliding with "+str(self.stop[0])+", "+str(self.stop[1]))

    """Get the final position of the piece, after the action"""
    def getFinalPos(self):
        if self.final_pos == ():
            if self.moving[0] == self.stop[0]:
                self.final_pos = (self.stop[0], self.stop[1] + (1 if self.stop[1] < self.moving[1] else -1))
            elif self.moving[1] == self.stop[1]:
                self.final_pos = (self.stop[0] + (1 if self.stop[0] < self.moving[0] else -1), self.stop[1])
            else:
                raise RuntimeError
        return self.final_pos

    """Get the cost of the action"""
    def actionCost(self):
        return abs(list(filter(lambda i: i != 0, tupleSub(self.moving, self.getFinalPos())))[0])


class PuzzleRobots(Problem):

    def __init__(self, initial, goal=None):
        self.initial = initial
        self.goal = goal

    def actions(self, state):
        for l in range(2):
            for i in range(state.size):
                col = [c for c in state.blacks + [state.white] if c[l] == i]
                col.sort()
                for a in range(len(col)):
                    if a != len(col)-1:
                        yield PuzzleRobotsAction(col[a], col[a+1])
                    if a != 0:
                        yield PuzzleRobotsAction(col[a], col[a-1])

    def result(self, state, action):
        new_state = state.clone()
        if(new_state.white == action.moving):
            new_state.white = action.getFinalPos()
        else:
            for i in range(len(new_state.blacks)):
                if(new_state.blacks[i] == action.moving):
                    new_state.blacks[i] = action.getFinalPos()
                    break
        return new_state
            
    def goal_test(self, state):
        return self.goal == state.white

    def path_cost(self, c, state1, action, state2):
        return c + action.actionCost()
    
    def display(self, state):
        state.display()

    def exec(self,estado,accoes):
        custo = 0
        for a in accoes:
            seg = self.result(estado,a)
            custo = self.path_cost(custo,estado,a,seg)
            estado = seg
            self.display(estado)
            print('Custo acumulado = '+custo)
            print('Ações possíveis:')
            for act in self.actions():
                act.display()
        return (estado,custo)


### Criação de  instâncias do problema e execução de sequências de acções
(Mostrem que o código está a funcionar, construindo instâncias da classe **PuzzleRobots**, fazendo o display dos estados, verificando o teste de estado final, gerando as acções para alguns estados, executando acções a partir de alguns estados e gerando novos estados e mostrando a evolução dos custos; verificando que os estados não se modificam com as acções (são gerados novos estados) e que a igualdade e a comparação entre estados funciona. Mostrem que a execução de sequências de acções está a funcionar bem.)

In [5]:
# Definição de instância do problema com N=5 com estado inicial representado no enunciado
init = PuzzleRobotsState(5, [(3,4), (0,2), (1,1), (3,1), (4,0)], (1,4))
prob = PuzzleRobots(init, (2,2))
actions1 = [PuzzleRobotsAction((1,4), (1,1)), PuzzleRobotsAction((3,4), (3,1)), PuzzleRobotsAction((1,2), (3,2))]
actions2 = [PuzzleRobotsAction((1,4), (3,4)), PuzzleRobotsAction((1,1), (3,1)), PuzzleRobotsAction((2,4), (2,1))]

#Estado não muda ações
initClone = init.clone()
prob.result(init)
print("initClone == init: " + initClone == init)

# Execução de uma sequência de acções até estado final com apresentação da evolução dos custos
print('Caminho 1:')
result1 = prob.exec(init, actions1)
print('É o objetivo: '+prob.goal_test(result1[0]))
print()

# Execução de uma outra sequência distinta de acções até estado final com apresentação da evolução dos custos
print('Caminho 2:')
result2 = prob.exec(init, actions2)
print('É o objetivo: '+prob.goal_test(result2[0]))

In [6]:
# Definição de instância do problema com N=7 com estado inicial ilustrado no início deste relatório
init = PuzzleRobotsState(7, [(0,1), (1,0), (2,5), (3, 2), (5,0), (4,3)], (5, 6))
prob = PuzzleRobots(init, (3,3))
actions3 = [PuzzleRobotsAction((5,6), (5,0)), PuzzleRobotsAction((1,0), (1,5)), PuzzleRobotsAction((5,1), (0,1)), PuzzleRobotsAction((1,1), (1,4)), PuzzleRobotsAction((1,3), (3,3))]
# Execução de uma sequência de acções até estado final com apresentação da evolução dos custos
result3 = prob.exec(init, actions3)
print('É o objetivo: '+prob.goal_test(result3[0]))

### Teste de Procura de Solução
(Quem quiser ir mais além, até pode confirmar que o problema está bem modelizado utilizando alguns dos algoritmos de procura como o **depth-first-search()** ou o **breadth-first-search()**, nas suas versões em árvore ou em grafo)

### Extras

(Se realizou o extra descreva aqui como abordou a sua representação em Python; pode acrescentar as células que considere convenientes)

In [7]:
class PuzzleRobotsGen(Problem):
    pass